In [43]:
import os
from os import path
from IPython.display import display, HTML
import json
import unicodecsv as csv
entity_dir = r'../data/ENTITY SET'

In [2]:
filenames = [path.join(entity_dir, fname) for fname in os.listdir(entity_dir)]
all_entities = []
for fname in filenames:
    with open(fname) as infile:
        content = unicode(infile.read(), 'gbk')
        all_entities.extend([i for i in content.split('\n') if len(i.strip()) != 0])
all_entities = set(all_entities)

In [3]:
print len(all_entities)
print ' '.join(list(all_entities)[:100])

140969
吴浩康 陆缘桂林米粉(厦门:思明区厦禾路) 八十天后说爱她(2010) 黑暗风暴(2006) 双头鹰之谜(1987) Frank Lounge(上海:杨浦区国庠路) 皖南私房菜馆(合肥:蜀山区望江路) 一品湘(宁波:北仑区小港街) 陈丽娟(歌仔戏演员) 陈文贵(东莞鱼光网络CEO) 人人都恨克里斯第二季(2006) 陈建(中国人民大学经济学教授) 破铁幕(1984) 亿食坊(广州:越秀区海珠中路) 和米堂(兰州:城关区甘南路) 爸爸出走了(2002) 张炜(著名影视制片人) 皇宫燕窝(上海:静安区石门一路) 吸血鬼3(2005) 梅耶荷德 四川姐弟肥肠鸡(沈阳:铁西区卫工南街) 控制游戏(2015) 关中刀客(2003) 金好牙 崔菲菲(《梅艳芳菲》参赛选手) 乾福·天津味道(天津:西青区华苑产业园区梅苑路) 王枝园农庄(广州:白云区钟落潭镇钟车路) 欧阳黔森 赫默尔(2012) 血战淞沪(2013) 程嘉美 王建军(青岛啤酒集团副总裁) 湘味蒸菜馆(上海:浦东新区唐镇吕三路) 客来家农家炒菜(青岛:李沧区李沧区秀峰路) 辣九香重庆老火锅(深圳:福田区福田区台湾花园东园路) 总统班底(1976) 雅谷花园餐厅酒吧(杭州:西湖区玉古路) 明晓溪(著名女作家) 麻辣小龙虾(沈阳:铁西区南六东路) 威尔斯(英国科幻作家) 南京灌汤包(济南:历下区山大路) 朝加真由美 王兰英(青海省委党校经济学部教授) 玖零后面馆(扬州:邗江区马太路) 李璞(陕西商州市城关中学教师) 李俐(贵州大学副教授) 美味乡村(上海:松江区江学路) 独一家美味烤鱼(武汉:江汉区民生路) 邻家小厨(宁波:北仑区小港公园路) 孟德(元朝昭勇大将军) 萨拉欢乐餐厅(上海:徐汇区桂林路) 赤松山魂(2006) 太学生话题社(2014) 葛丽亚·嘉逊 爱客仕(长春:朝阳区桂林路) 满铭焖鲜汇(扬州:广陵区靖泰淘宝) 四川小馆(深圳:罗湖区罗湖区太宁路) 杨铭宇黄焖鸡米饭(合肥:蜀山区望江西路) 锄奸记(1944) 杨军(德州市委常委) 卧底硬汉(2004) 八仙桌(成都:金堂五凤溪古镇) 京一舫海鲜(北京:朝阳区朝阳路) 李皇后(成汉国归义侯李势皇后) 张强(汪涵模仿者) 魔鬼征服者(1997) 老曾记面馆(昆明:西山区环城南路) 刘华杰(河北大学生命科学学院教授) 靓女阿萍(1985) 史塔克

In [4]:
sub_tasks = ['celebrity', 'movie', 'restaurant', 'tvShow']
trainset_locs = map(path.abspath, ['../data/TRAIN SET/%s.TRAINSET.txt' % t for t in sub_tasks])
devset_locs = map(path.abspath, ['../data/DEV SET/%s.DEVSET.txt' % t for t in sub_tasks])
output_locs = map(path.abspath, ['../output/%s.txt' % t for t in sub_tasks])
def LoadInData(data_loc, test_data=False):
    lines = unicode(open(data_loc).read(), 'gbk').split('\n')
    parsing_result = []
    for line in lines:
        terms = line.split('\t')
        items = []
        for i in terms[1:]:
            if test_data:
                ent, score = i, None

            else:
                colon_separated = i.split(':')
                ent = ':'.join(colon_separated[:-1])
                score = int(colon_separated[-1])
            items.append((ent, score))
        if len(items) == 0:
            continue
        parsing_result.append((terms[0], items))
    return parsing_result

train_data = map(LoadInData, trainset_locs)
dev_data = [LoadInData(d, True) for d in devset_locs]

In [5]:
used_in_cur_data = set()
for data_block in train_data + dev_data:
    for ent in data_block:
        for r, g in ent[1]:
            used_in_cur_data.add(r)

In [6]:
print len(used_in_cur_data)

96126


In [7]:
print set(used_in_cur_data) - set(all_entities)

set([])


# Searching on Baidu


In [8]:
import urllib
from bs4 import BeautifulSoup

def SearchForEntityName(name):
    request_template = 'http://baike.baidu.com/search?%s'
    #urllib.urlopen('')
    args = urllib.urlencode({"word" : name.encode('utf8'), 'pn' : 0, 'rn' : 0, 'enc' : 'utf8'})
    request = request_template % args

    response = urllib.urlopen(request).read()
    return unicode(response, 'utf8')
response = SearchForEntityName(list(all_entities)[0])

In [9]:
soup = BeautifulSoup(response)

for result in soup.find_all('dd'):
    try:
        title = result.find_all('a', {'class' : 'result-title'})[0]
        summary = result.find_all('p', {'class' : 'result-summary'})[0]
        print title.text.replace(u'_百度百科', ''), title['href'], summary.text
    except IndexError:
        continue
        

def ExtractFirstDesc(response):
    soup = BeautifulSoup(response)
    result = soup.find_all('dd')[0]
    try:
        title = result.find_all('a', {'class' : 'result-title'})[0]
        summary = result.find_all('p', {'class' : 'result-summary'})[0]
        return {'title' : title.text.replace(u'_百度百科', ''), 'link' : title['href'], 'summary' : summary.text}
    except IndexError:
        raise

print ' '.join(ExtractFirstDesc(response).values())

吴浩康 http://baike.baidu.com/view/109039.htm 吴浩康，1983年6月13日出生于香港，歌手。2002年参加无线电视自办的全球华人新秀赛总选获季军，开始出唱片、拍广告、电视剧和电影，曾因藏毒、殴打等...
上帝保佑 http://baike.baidu.com/subview/6400874/16588146.htm 1上帝保佑窦唯歌曲 ▪ 基本信息 ▪ 歌曲介绍 ▪ 歌曲歌词 2上帝保佑吴浩康、梁洛施歌曲 ▪ 歌曲信息 ▪ 歌词 ▪ 歌手简介 1...
囡 http://baike.baidu.com/view/256344.htm 由毕国智执导的影片《囡囡》,2010年04月05日在香港朗豪坊举行首映礼,主演林钰轩、冼色丽、诗雅、王敏奕、曾国祥、吴浩康、骆振伟等悉数到齐。该片讲述了几位援交...
严励行 http://baike.baidu.com/view/2212400.htm 吴浩康 - 惩罚自己(编) 吴浩康 - 真人Show(编) 吴浩康 - 前戏(编) 吴浩康 - 我可以(编) 泳儿- 黑眼圈(编) 方力申 - 脆弱的石头(编) 方力申 - 先苦后...
洗剪吹(吴浩康演唱歌曲) http://baike.baidu.com/subview/9684540/11032376.htm 《洗剪吹》是吴浩康演唱歌曲,所属专辑《黄伟文-Y100精选》。...... 《洗剪吹》是吴浩康演唱歌曲,所属专辑《黄伟文-Y100精选》。 中文名称 洗剪吹 所属专辑 《黄伟文...
爱不疚 http://baike.baidu.com/view/2638879.htm 时间歌曲歌手专辑 2008年12月 爱不疚 吴浩康 《Break It》[13] 参考资料 1. 组图:十大中文金曲颁奖礼 林峰演唱《爱不疚》 .腾讯娱乐[引用日期2014-05-5] 2....
关智斌 http://baike.baidu.com/view/91003.htm 迈向梦想的天空 (与吴浩康、吴卓羲合唱,电视剧《学警出更》主题曲) 梦幻组合 (与容祖儿twins郑希怡蒋雅文李逸朗 吴浩康 泳儿文彼得合唱) 死性不改 (boy'z时期与...
罪人(吴浩康演唱歌曲) http://baike.baidu.com/subview/550266/124813

In [10]:
import sqlite3

conn = sqlite3.connect('../entities_db/entities.db')
cursor = conn.cursor()

In [11]:
def TryParsingResposne((entity_name, response, _)):
    try:
        ExtractFirstDesc(response)
        return (1, 0)
    except:
        return (0, 1)

total_s, total_f = 0, 0
for i, item in enumerate(cursor.execute('select * from search_results limit 1000')):
    s, f = TryParsingResposne(item)
    total_s += s
    total_f += f
print total_s, total_f

849 151


In [26]:
failure_items = []
for i, item in enumerate(cursor.execute('select * from search_results limit 1000')):
    s, f = TryParsingResposne(item)
    if f:
        failure_items.append(item)
        if len(failure_items) == 10:
            break

In [32]:
display(HTML(failure_items[4][1]))

In [34]:
parsing_result = []
cursor.execute("select count(*) from search_results")
(total_amt, ) = cursor.fetchone()
for i, (entity_name, response, _) in enumerate(cursor.execute('select * from search_results')):
    if i % 1000 == 0:
        print "finished %d, out of %d" % (i, total_amt)
    try:
        info = ExtractFirstDesc(response)
    except:
        info = {}
    info['entity_name'] = entity_name
    parsing_result.append(info)

finished 0, out of 140969
finished 1000, out of 140969
finished 2000, out of 140969
finished 3000, out of 140969
finished 4000, out of 140969
finished 5000, out of 140969
finished 6000, out of 140969
finished 7000, out of 140969
finished 8000, out of 140969
finished 9000, out of 140969
finished 10000, out of 140969
finished 11000, out of 140969
finished 12000, out of 140969
finished 13000, out of 140969
finished 14000, out of 140969
finished 15000, out of 140969
finished 16000, out of 140969
finished 17000, out of 140969
finished 18000, out of 140969
finished 19000, out of 140969
finished 20000, out of 140969
finished 21000, out of 140969
finished 22000, out of 140969
finished 23000, out of 140969
finished 24000, out of 140969
finished 25000, out of 140969
finished 26000, out of 140969
finished 27000, out of 140969
finished 28000, out of 140969
finished 29000, out of 140969
finished 30000, out of 140969
finished 31000, out of 140969
finished 32000, out of 140969
finished 33000, out of 

In [44]:
with open('../entities_db/baike.csv', 'w') as ofile:
    writer = csv.DictWriter(ofile, ['entity_name', 'title', 'link', 'summary'])
    writer.writeheader()
    for row in parsing_result:
        writer.writerow(row)